# Importing libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/StanceCat-COV19

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/StanceCat-COV19


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import nltk
from tqdm import tqdm
from sklearn.metrics import f1_score
nltk.download('punkt')
from torch.optim import Adam

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data PreProcessing

In [ ]:
train = pd.read_csv('Dataset/train.csv')
text_train, category_train, stance_train = train['text'], train['category'], train['stance']

dev = pd.read_csv('Dataset/dev.csv')
text_dev, category_dev, stance_dev = dev['text'], dev['category'], dev['stance']

In [ ]:
train.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [ ]:
dev.head(10)

,text,category,stance
0,#مريم_رجوي: <LF>حظر خامنئي المجرم شراء #لقاح_ك...,info_news,1
1,#الصحة:<LF>•تم إعطاء 259.530 جرعة من لقاح #كور...,plan,1
2,#خادم_الحرمين - حفظه الله - يتلقى الجرعة الأول...,celebrity,1
3,#الصحه_العالميه: لقاحات #كورونا آمنة ولا خوف م...,info_news,1
4,"#وزيرة_الصحة ""#هالة_زايد"" تقول إنه يجرى مراجعة...",info_news,1
5,2️⃣ وانتهى الفريق من الدراسات قبل السريرية ونش...,info_news,1
6,عاجل 🔴 <LF>.<LF><LF>.<LF><LF>وزارة الصحة :<LF>...,plan,1
7,#فيديو | السفير الأميركي لدى السعودية بعد تلقي...,info_news,1
8,تصريحات وبس الحكومة مع السيسي علي حسب اللقطة! ...,info_news,0
9,الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا م...,info_news,1


In [ ]:
text_train, category_train, stance_train = np.array(train['text']), np.array(train['category']), np.array(train['stance'])
text_dev, category_dev, stance_dev = np.array(dev['text']), np.array(dev['category']), np.array(dev['stance'])

print(text_train.shape, category_train.shape, stance_train.shape)
print(text_dev.shape, category_dev.shape, stance_dev.shape)

(6988,) (6988,) (6988,)
(1000,) (1000,) (1000,)


In [ ]:
!pip install arabert
!pip install transformers
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_name="aubmindlab/bert-base-arabertv02-twitter"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text)
  
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")
model = AutoModelForMaskedLM.from_pretrained("aubmindlab/bert-base-arabertv02-twitter")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
example_text = text_train[0]
bert_input = tokenizer(example_text, padding='max_length', max_length = 100, truncation=True, return_tensors="pt")

print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

output = model(**bert_input)
print(output.logits.shape)

tensor([[    2,  6941, 28652, 16136, 34495,    10, 17448,   196, 13593,   306,
           650,  5166, 48342,   197,   139,   391,  5787,  7325,   197,   139,
           391, 11474,   139,  7829,   213,  3616,   185, 54749,   305,  2858,
         10639,   139,  1721,   338,  2779, 17613,  1686,   298,  5263,     1,
           634,   889, 13591,  5175,   323, 28892,    20,    20,    20,  9110,
           394,   418,  3682, 28892,  1422,   418,  6843, 36720,   306,  5263,
         62279,    77,   221,   221,   178,   223,    31,     1,     1,    89,
            20,    72,   219,     1,    54,   275,   272,   244,   244,   270,
           260,   264,   268,   179,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
categories = set(category_train)
print(categories)
category2id = {word:i for i, word in enumerate(list(categories))}
print(category2id['celebrity'])

{'requests', 'plan', 'advice', 'unrelated', 'info_news', 'celebrity', 'rumors', 'others', 'restrictions', 'personal'}
5


# Model Building

## BERT

### Ideas to try
1) bi-directional
2) pre-training
3) multi-layers
4) BERT
5) transformers notebook
6) packed_padded_sequences
7) pre-trained embedding

### Building Model

In [ ]:
class Dataset(torch.utils.data.Dataset):

  def __init__(self, x, y):

    x = x.copy()

    x = [arabert_prep.preprocess(text) for text in x]
    self.X = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in x]

    self.Y = torch.tensor(y)

    self.len = len(x)

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

In [ ]:
stance_train_dataset = Dataset(text_train, stance_train + 1)
category_train_dataset = Dataset(text_train, [category2id[category] for category in category_train])

stance_dev_dataset = Dataset(text_dev, stance_dev + 1)
category_dev_dataset = Dataset(text_dev, [category2id[category] for category in category_dev])

In [ ]:
from transformers import BertModel
class BertClassifier(nn.Module):

    def __init__(self, output_dim, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('aubmindlab/bert-base-arabertv02-twitter')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, output_dim)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Training

In [ ]:
def train(model, train_dataset, train_dataloader, criterion, optimizer):

  total_acc_train = 0
  total_loss_train = 0

  # for f1 score
  y_true, y_pred = [], []

  for train_input, train_label in tqdm(train_dataloader):

    # print("input_ids",train_input['input_ids'].shape)
    # print("attention_mask",train_input['attention_mask'].shape)

    train_label = train_label.to(device)
    mask = train_input['attention_mask'].to(device)
    input_id = train_input['input_ids'].squeeze(1).to(device)

    output = model(input_id, mask)

    # print(output.shape)

    batch_loss = criterion(output, train_label.long())
    total_loss_train += batch_loss.item()

    acc = (output.argmax(dim=1) == train_label).sum().item()
    total_acc_train += acc

    y_true += train_label.tolist() 
    y_pred += output.argmax(dim=1).tolist()

    model.zero_grad()
    batch_loss.backward()
    optimizer.step()

  # print(len(y_true), len(y_pred))
  f1_macro_train = f1_score(y_true, y_pred, average='macro')

  return total_loss_train/len(train_dataset), total_acc_train/len(train_dataset), f1_macro_train

In [ ]:
def evaluate(model, test_dataset, test_dataloader, criterion):

  total_acc_test = 0
  total_loss_test = 0

  # for f1 score
  y_true, y_pred = [], []

  with torch.no_grad():

    for test_input, test_label in test_dataloader:

      test_label = test_label.to(device)

      # print(test_input['input_ids'].shape)
      # print("attention_mask",train_input['attention_mask'].shape)

      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)

      output = model(input_id, mask)

      # print(output.shape)

      batch_loss = criterion(output, test_label.long())
      total_loss_test += batch_loss.item()

      acc = (output.argmax(dim=1) == test_label).sum().item()
      total_acc_test += acc

      y_true += test_label.tolist() 
      y_pred += output.argmax(dim=1).tolist()

  # print(len(y_true)-1, len(y_pred)-1)
  f1_macro_test = f1_score(y_true, y_pred, average='macro')

  return total_loss_test/len(test_dataset), total_acc_test/len(test_dataset), f1_macro_test

In [ ]:
def train_evaluate(model, train_dataset, val_dataset, learning_rate, epochs, model_name):

  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
  val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")

  criterion = nn.CrossEntropyLoss()
  optimizer = Adam(model.parameters(), lr= learning_rate)

  if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()

  best_f1_macro = 0

  for epoch_num in range(epochs):

    loss_train, acc_train, f1_macro_train = train(model, train_dataset, train_dataloader, criterion, optimizer)
    loss_val, acc_val, f1_macro_val = evaluate(model, val_dataset, val_dataloader, criterion)
          
    if f1_macro_val > best_f1_macro:
      best_f1_macro = f1_macro_val
    torch.save(model.state_dict(), f'Models/BERT/{model_name}/F1{f1_macro_val: .4f} Acc{acc_val: .4f}.pt')

    print(
        f'Epochs: {epoch_num + 1} \
        | Train Loss: {loss_train: .4f} \
        | Train Accuracy: {acc_train : .4f} \
        | Train F1_macro: {f1_macro_train: .4f} \
        | Val Loss: {loss_val: .4f} \
        | Val Accuracy: {acc_val: .4f} \
        | Val F1_macro: {f1_macro_val: .4f} \
        ')

In [ ]:
EPOCHS = 5
OUTPUT_DIM = 3
LR = 1e-6

model = BertClassifier(OUTPUT_DIM)
              
train_evaluate(model, stance_train_dataset, stance_dev_dataset, LR, EPOCHS, 'Stance')

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Epochs: 1         | Train Loss:  0.2729         | Train Accuracy:  0.7909         | Train F1_macro:  0.3660         | Val Loss:  0.2185         | Val Accuracy:  0.8170         | Val F1_macro:  0.4347         


100%|██████████| 3494/3494 [12:45<00:00,  4.57it/s]


Epochs: 2         | Train Loss:  0.2052         | Train Accuracy:  0.8240         | Train F1_macro:  0.4805         | Val Loss:  0.2032         | Val Accuracy:  0.8180         | Val F1_macro:  0.4936         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 3         | Train Loss:  0.1780         | Train Accuracy:  0.8495         | Train F1_macro:  0.5832         | Val Loss:  0.1990         | Val Accuracy:  0.8310         | Val F1_macro:  0.5690         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 4         | Train Loss:  0.1561         | Train Accuracy:  0.8745         | Train F1_macro:  0.6625         | Val Loss:  0.2036         | Val Accuracy:  0.8300         | Val F1_macro:  0.5731         


100%|██████████| 3494/3494 [12:45<00:00,  4.57it/s]


Epochs: 5         | Train Loss:  0.1275         | Train Accuracy:  0.9048         | Train F1_macro:  0.7633         | Val Loss:  0.2184         | Val Accuracy:  0.8300         | Val F1_macro:  0.6149         


In [ ]:
EPOCHS = 5
OUTPUT_DIM = 10
LR = 1e-6

model = BertClassifier(OUTPUT_DIM)
              
train_evaluate(model, category_train_dataset, category_dev_dataset, LR, EPOCHS, 'Category')

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Epochs: 1         | Train Loss:  0.6729         | Train Accuracy:  0.5944         | Train F1_macro:  0.1758         | Val Loss:  0.5104         | Val Accuracy:  0.7050         | Val F1_macro:  0.2333         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 2         | Train Loss:  0.5043         | Train Accuracy:  0.6895         | Train F1_macro:  0.2525         | Val Loss:  0.4733         | Val Accuracy:  0.7120         | Val F1_macro:  0.2630         


100%|██████████| 3494/3494 [12:45<00:00,  4.56it/s]


Epochs: 3         | Train Loss:  0.4613         | Train Accuracy:  0.7109         | Train F1_macro:  0.2850         | Val Loss:  0.4639         | Val Accuracy:  0.7110         | Val F1_macro:  0.2675         


100%|██████████| 3494/3494 [12:45<00:00,  4.57it/s]


Epochs: 4         | Train Loss:  0.4290         | Train Accuracy:  0.7293         | Train F1_macro:  0.3080         | Val Loss:  0.4634         | Val Accuracy:  0.7120         | Val F1_macro:  0.2842         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 5         | Train Loss:  0.3934         | Train Accuracy:  0.7523         | Train F1_macro:  0.3322         | Val Loss:  0.4729         | Val Accuracy:  0.6990         | Val F1_macro:  0.2844         


### continue Category

In [ ]:
train_evaluate(model, category_train_dataset, category_dev_dataset, LR, EPOCHS, 'Category')

100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 1         | Train Loss:  0.3561         | Train Accuracy:  0.7772         | Train F1_macro:  0.3558         | Val Loss:  0.4803         | Val Accuracy:  0.6910         | Val F1_macro:  0.2865         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 2         | Train Loss:  0.3163         | Train Accuracy:  0.8080         | Train F1_macro:  0.3857         | Val Loss:  0.5039         | Val Accuracy:  0.6530         | Val F1_macro:  0.2941         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 3         | Train Loss:  0.2743         | Train Accuracy:  0.8343         | Train F1_macro:  0.4312         | Val Loss:  0.5184         | Val Accuracy:  0.6750         | Val F1_macro:  0.2835         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 4         | Train Loss:  0.2347         | Train Accuracy:  0.8618         | Train F1_macro:  0.4790         | Val Loss:  0.5579         | Val Accuracy:  0.6400         | Val F1_macro:  0.3039         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 5         | Train Loss:  0.1997         | Train Accuracy:  0.8857         | Train F1_macro:  0.5333         | Val Loss:  0.5774         | Val Accuracy:  0.6650         | Val F1_macro:  0.3230         


### continue Stance

In [ ]:
EPOCHS = 5
OUTPUT_DIM = 3
LR = 1e-6

model = BertClassifier(OUTPUT_DIM)
state = torch.load('Models/BERT/Stance/F1 0.6149 Acc 0.8300.pt')
model.load_state_dict(state)
              
train_evaluate(model, stance_train_dataset, stance_dev_dataset, LR, EPOCHS, 'Stance')

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02-twitter were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02-twitter and are newly initialized: ['bert.pooler.dens

Epochs: 1         | Train Loss:  0.0977         | Train Accuracy:  0.9315         | Train F1_macro:  0.8319         | Val Loss:  0.2343         | Val Accuracy:  0.8300         | Val F1_macro:  0.6209         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 2         | Train Loss:  0.0743         | Train Accuracy:  0.9528         | Train F1_macro:  0.8848         | Val Loss:  0.2565         | Val Accuracy:  0.8350         | Val F1_macro:  0.6362         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 3         | Train Loss:  0.0568         | Train Accuracy:  0.9664         | Train F1_macro:  0.9188         | Val Loss:  0.2794         | Val Accuracy:  0.8300         | Val F1_macro:  0.6506         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 4         | Train Loss:  0.0434         | Train Accuracy:  0.9748         | Train F1_macro:  0.9412         | Val Loss:  0.3093         | Val Accuracy:  0.8120         | Val F1_macro:  0.6030         


100%|██████████| 3494/3494 [12:44<00:00,  4.57it/s]


Epochs: 5         | Train Loss:  0.0355         | Train Accuracy:  0.9784         | Train F1_macro:  0.9493         | Val Loss:  0.3311         | Val Accuracy:  0.8260         | Val F1_macro:  0.6018         
